In [ ]:
import collections
import glob
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


from scipy import stats

dataset_dir = "../data_prep/final_dataset/"

SUBSETS = "train dev test".split()

datasets = collections.defaultdict(list)

for subset in SUBSETS:
    for filename in glob.glob(dataset_dir + subset + "/*"):
        with open(filename, 'r') as f:
            datasets[subset].append(json.load(f))
            
all_pairs = sum(datasets.values(), [])

DPI = 150


FigText = collections.namedtuple("c", "x y text ha va rotation".split())
plt.rcParams.update({'font.size':'12',
                     'hatch.color': 'w'})

In [ ]:
palette = sns.color_palette("Dark2")
OVERALL_COLOR, REQUEST_COLOR, EVAL_COLOR, FACT_COLOR, STRUCT_COLOR = palette[:5]
AUX_COLOR = palette[-2]
sns.palplot(palette)

In [ ]:
spearmen = []
for pair in all_pairs:
    alignment_list = []
    for sentence in pair["rebuttal_sentences"]:
        align_type, align_indices = sentence["alignment"]
        if align_type == "context_sentences":
            for review_index in align_indices:
                alignment_list.append([sentence["sentence_index"], review_index])
    if not alignment_list:
        continue
    a, b = zip(*alignment_list)
    if len(set(a)) == 1 or len(set(b)) == 1:
        continue
    spearmen.append(stats.spearmanr(*zip(*alignment_list)).correlation)
    

plt.figure(figsize=(12, 4), dpi=DPI)
fig = sns.histplot(spearmen, bins=100, color=OVERALL_COLOR)
plt.ylabel("Number of review-rebuttal pairs")
plt.xlabel("Spearman's rho (rebuttal sent. idxs v/s aligned review sent. idxs)")

plt.savefig("figs/spearmen.png")



In [ ]:
review_sentence_counter = {}

for pair in all_pairs:
    review_id = pair["metadata"]["review_id"]
    review_sentence_counter[review_id] = {
        "mapped": [False]* len(pair["review_sentences"]),
        "arg_type": [sentence["coarse"] for sentence in pair["review_sentences"]]}
    for rebuttal_sentence in pair["rebuttal_sentences"]:
        align_type, aligned = rebuttal_sentence["alignment"]
        if align_type == "context_sentences":
            for index in aligned:
                try:
                    review_sentence_counter[review_id]["mapped"][index] = True
                except IndexError:
                    pass
                
color_list = [OVERALL_COLOR, REQUEST_COLOR, EVAL_COLOR, FACT_COLOR]

fig, axs = plt.subplots(4, 1, figsize=(12,5), sharex=True, dpi=DPI)

coverages = []
for review_id, mappyloo in review_sentence_counter.items():
    coverages.append(sum(mappyloo["mapped"])/len(mappyloo["mapped"]))
sns.histplot(coverages, ax=axs[0], bins=20, color=color_list[0])

axs[0].yaxis.label.set_visible(False)

for i, arg_type in enumerate("overall request evaluative fact".split()):
    if i == 0:
        continue
    actual_arg_type = "arg_" + arg_type

    coverages = []
    for review_id, mappyloo in review_sentence_counter.items():
        filtered = [mappyloo["mapped"][j] for j, t in enumerate(mappyloo["arg_type"]) if t == actual_arg_type]
        if not filtered:
            continue
        coverages.append(sum(filtered)/len(filtered))


    sns.histplot(coverages, ax=axs[i], bins=20, color=color_list[i])

    
y_axis_label = FigText(0.07, 0.5, "Number of reviews", "left", "center", "vertical")
x_axis_label = FigText(0.5, 0.01,
                       'Proportion of review sentences with direct responses, by action type',
                       "center", "bottom", "horizontal")
plot_labels = [
    x_axis_label, y_axis_label,
    FigText(0.75, 0.62, 'Request', "left", "center", "horizontal"),
    FigText(0.75, 0.43, 'Evaluative', "left", "center", "horizontal"),
    FigText(0.75, 0.22, 'Fact', "left", "center", "horizontal"),
    FigText(0.75, 0.82, 'All actions', "left", "center", "horizontal"),
]


for ft in plot_labels:
    fig.text(ft.x, ft.y, ft.text, ha=ft.ha, va=ft.va, rotation=ft.rotation)

for i in range(4):
    axs[i].spines['top'].set_visible(False)
    axs[i].spines['right'].set_visible(False)
    axs[i].yaxis.label.set_visible(False)


plt.savefig("figs/coverage.png")

In [ ]:
# Agreeability v/s variance

import numpy as np

def get_agreeability(pair_obj):
    coarse_counter = collections.Counter()
    for sentence in pair_obj["rebuttal_sentences"]:
        coarse_counter[sentence['coarse']] += 1
    if 'concur' not in coarse_counter and 'dispute' not in coarse_counter:
        return {'agreeability': None}
    return {
      "agreeability":
        coarse_counter['concur'] / (coarse_counter['concur'] + coarse_counter['dispute'])
    }

def draw_constant_line(value, other_axis_values, is_horizontal):
    pass

forum_agreeability_map = collections.defaultdict(list)

for pair in all_pairs:
    forum_agreeability_map[pair["metadata"]["forum_id"]].append((get_agreeability(pair)["agreeability"],pair["metadata"]["rating"] ))

agreeability_df_dicts = []


# Column names
agree_mean, rating_var, avg_rating = ["Mean agreeability score", "Variance in rating across reviewers", "Average rating"]
    
for forum, info in forum_agreeability_map.items():
    if len(info) == 1:
        continue
    agreeabilities, ratings = list(zip(*info))
    if None in agreeabilities:
        continue
    agreeability_df_dicts.append({agree_mean: np.mean(agreeabilities),
                                 rating_var: np.var(ratings),
                                 avg_rating: np.mean(ratings)})
    
agreeability_df = pd.DataFrame.from_dict(agreeability_df_dicts)

agreeability_first_quartile = agreeability_df.quantile(0.25)[agree_mean]
variance_third_quartile = agreeability_df.quantile(0.75)[rating_var]


plt.figure(figsize=(12, 6), dpi=DPI)
ax = sns.scatterplot(
    data=agreeability_df,
    x=agree_mean,
    y=rating_var,
    hue=avg_rating, s=100,palette=sns.light_palette(OVERALL_COLOR, as_cmap=True))
plt.plot([agreeability_first_quartile,agreeability_first_quartile], [0,8.5], linestyle="dashed", color=AUX_COLOR)
plt.plot([0,1], [variance_third_quartile,variance_third_quartile], linestyle="dotted", color=AUX_COLOR)


plot_labels = [
    FigText(0.5, 9.4, 'Authors agree with reviewers more', "left", "center", "horizontal"),
    FigText(1.06, 5, "Reviewers agree with each other more", "right", "center", 270)
]


ax.arrow(0.3, 9.1, 0.65,0, head_width=0.25, head_length=0.03, fc='k', ec='k')

ax.arrow(1.03, 9, 0, -8, head_width=0.018, head_length=0.4, fc='k', ec='k')


ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


for ft in plot_labels:
    plt.text(ft.x, ft.y, ft.text, ha=ft.ha, va=ft.va, rotation=ft.rotation)
    

plt.tight_layout()
plt.savefig("figs/agreeability.eps")

In [ ]:

j = collections.defaultdict(lambda:collections.Counter())

for example in all_pairs:
    review_coarse_labels = [sentence["coarse"] for sentence in example["review_sentences"]]
    for rebuttal_sentence in example["rebuttal_sentences"]:
        fine = rebuttal_sentence["fine"]
        align_type, aligned_idxs = rebuttal_sentence["alignment"]
        if align_type == "context_sentences":
            for aligned_idx in rebuttal_sentence["alignment"][1]:
                try:
                    j[fine][review_coarse_labels[aligned_idx]] += 1
                except IndexError:
                    pass

review_types = "arg_request arg_evaluative arg_fact arg_structuring arg_social arg_other".split()

type_name_map = {
    "arg_request": "Request",
    "arg_evaluative": "Evaluative",
    "arg_fact": "Fact",
    "arg_social": "Social",
    "arg_structuring": "Structuring",
    "arg_other": "Other",
}

full_eval_responses = [
    "rebuttal_accept-praise",
"rebuttal_concede-criticism",
"rebuttal_mitigate-criticism",
"rebuttal_reject-criticism",
]

full_request_responses = [
    "rebuttal_answer",
"rebuttal_by-cr",
"rebuttal_done",
"rebuttal_future",
"rebuttal_refute-question",
"rebuttal_reject-request",
]

eval_responses = [
"rebuttal_concede-criticism",
]

request_responses = [
    "rebuttal_answer",
"rebuttal_done",
]



fig, axes = plt.subplots(nrows=len(eval_responses + request_responses), ncols=1, figsize=(12,6), dpi=DPI)

k=sns.color_palette("crest", 2)


ax_count = 0
for key in sorted(eval_responses) + sorted(request_responses):
    vals = j[key]
    bar = sns.barplot(x=[type_name_map[i] for i in review_types], y=[vals[i] for i in review_types],ax=axes[ax_count], palette=[REQUEST_COLOR, EVAL_COLOR, FACT_COLOR, STRUCT_COLOR, OVERALL_COLOR, OVERALL_COLOR])
    
    if key in eval_responses:
        bar.patches[1].set_hatch("/")
    elif key in request_responses:
        bar.patches[0].set_hatch("/")
    else:
        assert False
    #bar.set_edgecolor([0.5,0.5,0.5])
    axes[ax_count].set_ylabel(key[9:])
    ax_count += 1
    

for i in range(3):
    axes[i].spines['top'].set_visible(False)
    axes[i].spines['right'].set_visible(False)
    axes[i].yaxis.label.set_visible(False)

plot_labels = [
    FigText(0.5, 0.8, 'Contexts of "Concede criticism" sentences', "left", "center", "horizontal"),
    FigText(0.5, 0.55, 'Contexts of "Answer" sentences', "left", "center", "horizontal"),
    FigText(0.5, 0.28, 'Contexts of "Task has been done" sentences', "left", "center", "horizontal"),
    FigText(0.06, 0.5, "Number of rebuttal sentences", "left", "center", "vertical")
]


for ft in plot_labels:
    fig.text(ft.x, ft.y, ft.text, ha=ft.ha, va=ft.va, rotation=ft.rotation)
    
    
plt.savefig("figs/noncanonical.png")